# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = 'drive/MyDrive/5. Spark/spark-scripts/section2/Datasets/'

Mounted at /content/drive


In [ ]:
!pip install pyspark
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('agg').getOrCreate()
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 25.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b75736b351457af7d7d092193de115d3a8fd0608b69fdb162f69cb89b2fe53f1
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


## Read in the dataFrame for this Notebook

In [ ]:
airbnb = spark.read.csv(path +'nyc_air_bnb.csv',inferSchema=True,header=True)

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking. 

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_rows',None)

airbnb.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Now print the schema so we can make sure all the variables have the correct types

In [ ]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
airbnb = airbnb.withColumn('price', airbnb.price.cast(IntegerType())) \
                .withColumn('minimum_nights', airbnb.minimum_nights.cast(IntegerType())) \
                .withColumn('number_of_reviews', airbnb.number_of_reviews.cast(IntegerType())) \
                .withColumn('reviews_per_month', airbnb.reviews_per_month.cast(FloatType())) \
                .withColumn('calculated_host_listings_count', airbnb.calculated_host_listings_count.cast(IntegerType())) \
                .withColumn('availability_365', airbnb.availability_365.cast(IntegerType())) 

In [ ]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: float (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)



### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [ ]:
airbnb.count()

49079

### 2. How many total reviews does each host have?

In [ ]:
airbnb.groupBy('host_id').sum('Number_of_reviews').show()

+-------+----------------------+
|host_id|sum(Number_of_reviews)|
+-------+----------------------+
| 716306|                   197|
|1203500|                    35|
| 368528|                     1|
|1577493|                    16|
|1390555|                    50|
|1317588|                     3|
|2472680|                   219|
|2155832|                   266|
|2426404|                     6|
|2740824|                    22|
| 192750|                     2|
|2682735|                    50|
|4432173|                    15|
|5959653|                   240|
|4623093|                    79|
|6390340|                     0|
|4645357|                   165|
|8422502|                     2|
|9818634|                     0|
|4906960|                    65|
+-------+----------------------+
only showing top 20 rows



### 3. Show the min and max of all the numeric variables in the dataset

In [ ]:
airbnb.select('price','Number_of_reviews','minimum_nights','reviews_per_month').summary('min','max').show()

+-------+-----+-----------------+--------------+-----------------+
|summary|price|Number_of_reviews|minimum_nights|reviews_per_month|
+-------+-----+-----------------+--------------+-----------------+
|    min|  -74|                0|             0|              0.0|
|    max|10000|              629|          1250|             58.0|
+-------+-----+-----------------+--------------+-----------------+



### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [ ]:
airbnb.groupBy('host_id').sum('Number_of_reviews').withColumnRenamed('sum(Number_of_reviews)','total_reviews') \
                          .orderBy(col('total_reviews').desc()).show(1)

+--------+-------------+
| host_id|total_reviews|
+--------+-------------+
|37312959|         2273|
+--------+-------------+
only showing top 1 row



### 5. On average, how many nights did most hosts specify for a minimum?

In [ ]:
airbnb.agg(avg('minimum_nights')).withColumnRenamed('avg(minimum_nights)','avg minimum nights').show()

+------------------+
|avg minimum nights|
+------------------+
|7.1286126280910596|
+------------------+



### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [ ]:
airbnb.groupBy('neighbourhood').agg(avg('price')) \
      .withColumnRenamed('avg(price)','avg_price').orderBy(col('avg_price').desc()).show(1)

+--------------+---------+
| neighbourhood|avg_price|
+--------------+---------+
|Fort Wadsworth|    800.0|
+--------------+---------+
only showing top 1 row



### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [ ]:
airbnb.groupBy("room_type").pivot("neighbourhood_group", ["Queens", "Brooklyn"]).agg(avg('price')) \
                           .filter("room_type == 'Private room' or  room_type == 'Shared room'").show(10)

+------------+-----------------+-----------------+
|   room_type|           Queens|         Brooklyn|
+------------+-----------------+-----------------+
| Shared room|69.02020202020202|50.52784503631961|
|Private room|71.77054078279056|76.47234042553191|
+------------+-----------------+-----------------+



### Alright that's all folks!

### Great job!